## Blender add-on notes

> Create add-on for blender to do mesh-creation from segmentation and pullbacks from within blender

In [18]:
from blender_tissue_cartography import io as tcio
from blender_tissue_cartography import mesh as tcmesh

from blender_tissue_cartography import interpolation as tcinterp
from blender_tissue_cartography import rotation as tcrot
from blender_tissue_cartography import diffgeo as tcdfg

import numpy as np
from copy import deepcopy
import warnings
import igl

from scipy import interpolate, ndimage, optimize, sparse, spatial
from skimage import registration, transform

import matplotlib as mpl
import matplotlib.pyplot as plt

### Experiment with the tutorial dataset

In [26]:
tutorial_mesh = tcmesh.ObjMesh.read_obj("Tutorials/drosophila_example/Drosophila_CAAX-mCherry_mesh_remeshed.obj")
tutorial_data = tcio.imread("Tutorials/drosophila_example/Drosophila_CAAX-mCherry.tif")
resolution_array = np.array([1.05, 1.05, 1.05])

In [27]:
tutorial_mesh.vertices.min(axis=0), tutorial_mesh.vertices.max(axis=0)

(array([0.        , 0.        , 7.15691042]),
 array([191.57263898, 523.00282661, 186.64725986]))

In [28]:
tutorial_data.shape

(190, 509, 188)

In [46]:
ls = np.linalg.norm(tutorial_mesh.vertices[tutorial_mesh.faces[:,0]] - tutorial_mesh.vertices[tutorial_mesh.faces[:,1]], axis=1)
anti_aliasing_scale = np.median(ls)/2
tutorial_data_smoothed = ndimage.gaussian_filter(tutorial_data, anti_aliasing_scale/resolution_array)

In [47]:
x, y, z = [np.arange(ni)/resolution_array[i] for i, ni in enumerate(tutorial_data.shape)]

interpolator = interpolate.RegularGridInterpolator((x,y,z), tutorial_data_smoothed, method='linear', bounds_error=False)

In [ ]:

    baked_data = []
    for o in normal_offsets:
        baked_layer_data = np.stack([interpolate.interpn((x, y, z), channel,
                                     (baked_world_positions+o*baked_normals)/resolution,
                                     method="linear", bounds_error=False) for channel in image])
        baked_data.append(baked_layer_data)
    baked_data = np.stack(baked_data, axis=1)

In [53]:
shape = (3, 10, 40, 40)
arr = np.random.normal(size=shape)
np.allclose(np.array(arr.tolist()).reshape(shape), arr)

True

In [56]:
arr = np.random.normal(size=(1000,1000,200))

In [57]:
%%time
ndimage.gaussian_filter(arr, sigma=(4,4,4))

CPU times: user 6.51 s, sys: 450 ms, total: 6.96 s
Wall time: 6.96 s


array([[[-3.39334310e-02, -3.25001619e-02, -3.01559763e-02, ...,
          3.01606432e-02,  3.84222384e-02,  4.29556282e-02],
        [-3.50301492e-02, -3.36526904e-02, -3.14067043e-02, ...,
          2.86426603e-02,  3.66980625e-02,  4.11168310e-02],
        [-3.69279141e-02, -3.56413538e-02, -3.35532689e-02, ...,
          2.59516585e-02,  3.36097662e-02,  3.78064303e-02],
        ...,
        [-2.20244096e-02, -2.18784777e-02, -2.15578943e-02, ...,
         -3.91593486e-02, -4.28806787e-02, -4.48153980e-02],
        [-1.89296449e-02, -1.88159862e-02, -1.85722334e-02, ...,
         -3.21406552e-02, -3.52243591e-02, -3.68219489e-02],
        [-1.72245335e-02, -1.71274879e-02, -1.69230168e-02, ...,
         -2.83505424e-02, -3.10802602e-02, -3.24907098e-02]],

       [[-3.26145738e-02, -3.13082272e-02, -2.91692788e-02, ...,
          2.55707444e-02,  3.33638298e-02,  3.76390111e-02],
        [-3.37027537e-02, -3.24474623e-02, -3.03983635e-02, ...,
          2.41603056e-02,  3.17499198e

<function scipy.ndimage._filters.gaussian_filter(input, sigma, order=0, output=None, mode='reflect', cval=0.0, truncate=4.0, *, radius=None, axes=None)>

In [40]:
tutorial_data.shape

(190, 509, 188)

### Notes and To do

1. Image normalization for ortho-slices and textures. Maybe add a global variable "normalized image"? Normalization slider in materials.

2. axis permutation

3. marching cubes

4. better color options for vertex shading


More generally: somehow associate the data (like projections etc) to the mesh, and the image data to the bounding box, instead of having a global variable floating around. This can be done by assigning custom data as

```
obj["data_name"] = (np.array(...), shape)

```
Need to reconstruct using `np.array(arr.tolist()).reshape(shape)`.

`bpy.context.selected_objects`.


Idea: selected + active objects. Make sure both bounding box and mesh are selected. Give the bounding box some attribute so I know which of the selected objects is the 3d data and which is the mesh.

## Specs

### Inputs

1. Path to `.tiff` file
2. $x$/$y$/$z$ resolution (can be read out automatically)
3. Optional: path to segmentation `.tiff`

The user can also import the mesh directly. 

### Functionality

Convention: mesh vertex coordinates always have to be in microns.

0. Create bounding box (in microns) in blender that shows outline of volumetric data. Allow loading orthoslices of raw data.

1. Upon loading, convert segmentation to mesh using volume to mesh. Options: smoothing scale.

Errors: file not found, segmentation and image do not have the same number of pixels. (Q: allow separate x/y/z resolution for segmentation?)

2. Upon button click, create multilayer pullback and save to disk as pngs and multi page tiff. Also save normal and 3d coord bake. As well as max projection.

**Options**: resolution (pixels), normal layer spacing.

**Errors**: file not found, no UV map

**Warnings**: UV map has self intersections, UV map out of bounds

### Internals

1. Meshing (segmentation -> mesh). Load segmentation as .tiff, convert to vdb volume, use blenders volume to mesh node

2. Cartographic projection. The mesh used is the currently selected one! Two steps.

    a. Bake 3d coordinates and normals to UV. Use `scipy`-interpolation - render bake is difficult to get to work.

    b. Bake from volumetric data to UV. Use `scipy`-interpolation as in the pytho package.

4. Saving. Always save to fixed path, say extracted_textures in the same path as image. Careful with windows, Linux, etc path handling.

5. Create textures/shader from baked data.

6. Load orthoslices of raw data

7. Optional: Create normal shifted meshes (not visible by default). Shrink/fatten blender button.

### Conventions:

1. Mesh orientation: $Y$ forward, $Z$ up! (when importing the mesh).

### Dependencies:

1. Numpy (standard)
2. Scipy, if available (interpolation). I can also do interpolation myself 
3. `tifffile` for reading `.tiff` files (check microscopy nodes source code), and converting to .vdb (for meshing) OR some marching cubes code.


TO DO: vertex color based shading! Bounding box!

`numpy`, `scipy`, `tifffile` and `skimage` are shipped with Blender's python!

## Ressources

Add-on Tutorial: https://docs.blender.org/manual/en/latest/advanced/scripting/addon_tutorial.html

MicroscopyNodes source code: https://github.com/oanegros/MicroscopyNodes

### Experiments

In [4]:
import tifffile

### Add-on code

In [ ]:
bl_info = {
    "name": "Tissue Cartography",
    "blender": (3, 0, 0),
    "category": "Scene",
}

import bpy
from bpy.props import StringProperty, FloatVectorProperty, FloatProperty, IntProperty
from bpy.types import Operator, Panel
from pathlib import Path
import numpy as np
import tifffile
from scipy import interpolate

### Functions for tissue cartography

def load_png(image_path):
    """Load .png into numpy array."""
    image = bpy.data.images.load(image_path)
    width, height = image.size
    pixels = np.array(image.pixels[:], dtype=np.float32)
    return pixels.reshape((height, width, -1))

def get_uv_layout(uv_layout_path, image_resolution):
    """Get UV layout mask for currently active object and save UV layout to disk"""
    bpy.ops.object.mode_set(mode='EDIT')
    bpy.ops.uv.export_layout(filepath=uv_layout_path, size=(image_resolution, image_resolution), opacity=1)
    bpy.ops.object.mode_set(mode='OBJECT')
    UV_layout = load_png(uv_layout_path)
    return (UV_layout.sum(axis=-1) > 0)[::-1]

def get_uv_normal_world_per_loop(mesh_obj, filter_unique=False):
    """
    Get UV, normals, and world and normal for each loop (half-edge) as np.array.
    
    If filter_unique, remove "duplicate" loops (for which UV, normals and position
    are identical).
    """
    if not mesh_obj:
        raise TypeError("No object selected")
    if mesh_obj.type != 'MESH':
        raise TypeError("Selected object is not a mesh")
    world_matrix = mesh_obj.matrix_world
    uv_layer = mesh_obj.data.uv_layers.active
    if not uv_layer:
        raise RuntimeError("Mesh does not have an active UV map")
    loop_uvs = np.zeros((len(mesh_obj.data.loops), 2), dtype=np.float32)
    loop_normals = np.zeros((len(mesh_obj.data.loops), 3), dtype=np.float32)
    loop_world_positions = np.zeros((len(mesh_obj.data.loops), 3), dtype=np.float32)
    for loop in mesh_obj.data.loops:
        loop_uvs[loop.index] = uv_layer.data[loop.index].uv
        loop_normals[loop.index] = world_matrix.to_3x3() @ mesh_obj.data.vertices[loop.vertex_index].normal
        loop_world_positions[loop.index] = world_matrix @ mesh_obj.data.vertices[loop.vertex_index].co
    if filter_unique:
        unqiue_loops = np.unique(np.hstack([loop_uvs, loop_normals, loop_world_positions]), axis=0)
        loop_uvs, loop_normals, loop_world_positions = (unqiue_loops[:,:2], unqiue_loops[:,2:5], unqiue_loops[:,5:])
    loop_normals = np.round((loop_normals.T/np.linalg.norm(loop_normals, axis=1)).T, decimals=4)
    return loop_uvs, loop_normals, loop_world_positions

def bake_per_loop_values_to_uv(loop_uvs, loop_values, image_resolution):
    """
    Bake (interpolate) values (normals or world position) defined per loop into the UV square.
    
    UV coordinates outside [0,1] are ignored.
    
    Parameters
    ----------
    loop_uvs : np.array of shape (n_loops, 2)
        UV coordinates of loop.
    loop_values : np.array of shape (n_loops, ...)
        Input field. Can be an array with any number of axes (e.g. scalar or vector field).
    image_resolution : int, default 256
        Size of UV grid. Determines resolution of result.

    Returns
    -------
    
    interpolated : np.array of shape (uv_grid_steps, uv_grid_steps, ...)
        Field across [0,1]**2 UV grid, with a uniform step size. UV positions that don't
        correspond to any value are set to np.nan.
            
    """
    U, V = np.meshgrid(*(2*(np.linspace(0,1, image_resolution),)))
    interpolated = interpolate.griddata(loop_uvs, loop_values, (U, V), method='linear')[::-1]
    return interpolated
    
def bake_volumetric_data_to_uv(image, baked_world_positions, resolution, baked_normals, normal_offsets=(0,)):
    """ 
    Interpolate volumetric image data onto UV coordinate grid.
    
    Uses baked 3d world positions corresponding to each UV grid point (see bake_per_loop_values_to_UV).
    3d coordinates (in microns) are converted into image coordinates via the resolution scaling factor.
    The resolution of the bake (number of pixels) is determined by the shape of baked_world_positions.
    
    normal_offsets moves the 3d positions whose volumetric voxel values will be baked inwards or outwards
    along the surface normal. Providing a list of offsets results in a multi-layer pullback
    
    Parameters
    ----------
    image : 4d np.array
        Image, axis 0  is assumed to be the channel axis
    baked_world_positions : np.array of shape (image_resolution, image_resolution, uv_grid_steps, 3)
        3d world positions baked to UV grid, with uniform step size. UV positions that don't correspond to 
        any value are set to np.nan.
    resolution : np.array of shape (3,)
        Resolution in pixels/microns for each of the three spatial axes.
    baked_normals : np.array of shape (image_resolution, image_resolution, uv_grid_steps, 3)
        3d world normals baked to UV grid, with uniform step size. UV positions that don't correspond to 
        any value are set to np.nan.
    normal_offsets : np.array of shape (n_layers,), default (0,)
        Offsets along normal direction, in same units as interpolated_3d_positions (i.e. microns).
        0 corresponds to no shift.
        
    Returns
    -------
    aked_data : np.array of shape (n_channels, n_layers, image_resolution, image_resolution)
        Multi-layer 3d volumetric data baked onto UV.
    """
    x, y, z = [np.arange(ni) for ni in image.shape[1:]]
    baked_data = []
    for o in normal_offsets:
        baked_layer_data = np.stack([interpolate.interpn((x, y, z), channel,
                                     (baked_world_positions+o*baked_normals)/resolution,
                                     method="linear", bounds_error=False) for channel in image])
        baked_data.append(baked_layer_data)
    baked_data = np.stack(baked_data, axis=1)
    return baked_data

### Operators defining the user interface of the add-on

class LoadTIFFOperator(Operator):
    """Load .tif file and resolution"""
    bl_idname = "scene.load_tiff"
    bl_label = "Load TIFF File"

    def execute(self, context):
        file_path = context.scene.tissue_cartography_file
        resolution = context.scene.tissue_cartography_resolution

        # Load resolution as a NumPy array
        resolution_array = np.array(resolution)
        self.report({'INFO'}, f"Resolution loaded: {resolution_array}")

        # Load TIFF file as a NumPy array
        if not (file_path.lower().endswith(".tiff") or file_path.lower().endswith(".tif")):
            self.report({'ERROR'}, "Selected file is not a TIFF")
            return {'CANCELLED'}
        try:
            data = tifffile.imread(file_path)
            if len(data.shape) == 3: # add singleton channel axis to single channel-data 
                data = data[np.newaxis]
            assert len(data.shape) == 4, "Data must be volumetric!"
            self.report({'INFO'}, f"TIFF file loaded with shape {data.shape}")
            # Store variables in Blender's global storage
            bpy.types.Scene.tissue_cartography_data = data
            bpy.types.Scene.tissue_cartography_resolution_array = resolution_array
        except Exception as e:
            self.report({'ERROR'}, f"Failed to load TIFF file: {e}")
            return {'CANCELLED'}

        return {'FINISHED'}


class CreateProjectionOperator(Operator):
    """
    Create a cartographic projection.
    
    This is done in two steps: first, bake 3d world positions and normals to UV,
    then use the baked positions to interpolate the volumetric data
    to UV.
    """
    bl_idname = "scene.create_projection"
    bl_label = "Create Projection"

    def execute(self, context):
        # Validate selected object and UV map
        obj = context.active_object
        if not obj or obj.type != 'MESH':
            self.report({'ERROR'}, "No mesh object selected!")
            return {'CANCELLED'}
        # Ensure the object has a UV map
        if not obj.data.uv_layers:
            self.report({'ERROR'}, "The selected mesh does not have a UV map!")
            return {'CANCELLED'}
        
        # Parse offsets into a NumPy array
        offsets_str = context.scene.tissue_cartography_offsets
        try:
            offsets_array = np.array([float(x) for x in offsets_str.split(",") if x.strip()])
            if offsets_array.size == 0:
                offsets_array = np.array([0])
            bpy.types.Scene.tissue_cartography_offsets_array = offsets_array
            self.report({'INFO'}, f"Offsets loaded: {offsets_array}")
        except ValueError as e:
            self.report({'ERROR'}, f"Invalid offsets input: {e}")
            return {'CANCELLED'}
        
        # Parse projection resolution
        projection_resolution = context.scene.projection_resolution
        self.report({'INFO'}, f"Using projection resolution: {projection_resolution}")

        # texture bake normals and world positions
        loop_uvs, loop_normals, loop_world_positions = get_uv_normal_world_per_loop(obj, filter_unique=True)
        baked_normals = bake_per_loop_values_to_uv(loop_uvs, loop_normals, image_resolution=projection_resolution)
        baked_normals = (baked_normals.T/np.linalg.norm(baked_normals.T, axis=0)).T
        baked_world_positions = bake_per_loop_values_to_uv(loop_uvs, loop_world_positions, image_resolution=projection_resolution)

        # obtain UV layout and use it to get a mask
        uv_layout_path = str(Path(bpy.path.abspath("//")).joinpath('UV_layout.png'))
        mask = get_uv_layout(uv_layout_path, projection_resolution)
        baked_normals[~mask] = np.nan
        baked_world_positions[~mask] = np.nan
        
        # create a pullback
        baked_data = bake_volumetric_data_to_uv(bpy.types.Scene.tissue_cartography_data,
                                                baked_world_positions, np.array([1,1,1]), baked_normals,
                                                normal_offsets=bpy.types.Scene.tissue_cartography_offsets_array)

        # set as global variables and save to disk
        bpy.types.Scene.tissue_cartography_baked_normals = baked_normals
        bpy.types.Scene.tissue_cartography_baked_world_positions = baked_world_positions
        bpy.types.Scene.tissue_cartography_baked_data = baked_data
        tifffile.imwrite(Path(bpy.path.abspath("//")).joinpath('BakedNormals.tif'), baked_normals)
        tifffile.imwrite(Path(bpy.path.abspath("//")).joinpath('BakedPositions.tif'), baked_world_positions)
        tifffile.imwrite(Path(bpy.path.abspath("//")).joinpath('BakedData.tif'), baked_data)
        self.report({'INFO'}, "Projected data saved to: "+bpy.path.abspath("//"))
        
        
        # [...]

        return {'FINISHED'}

class HelpPopupOperator(Operator):
    """Show help window."""
    bl_idname = "scene.help_popup"
    bl_label = "Tissue Cartography Help"

    def execute(self, context):
        return context.window_manager.invoke_popup(self, width=400)

    def draw(self, context):
        layout = self.layout
        col = layout.column()

        col.label(text="Tissue Cartography Add-On Help", icon='INFO')
        col.label(text="1. Load a .tiff file using the 'Load' button.")
        col.label(text="   after entering the resolution (x, y, z) in microns.")
        col.label(text="2. Select the mesh to use (it should be outlined in orange).")
        col.label(text="3. Click 'Create Projection' to bake mesh positions,")
        col.label(text="   nesh normals, and volumetric data to UV textures.")
        col.label(text="   Define normal offsets to get a multi-layer projection.")
        col.label(text="   Data is saved as .tiff files for further processing,")
        col.label(text="   and as a material to shade the mesh (see Shading workspace).")
        col.label(text="   Projection resolution determines the output texture size.")
        col.separator()
        col.label(text="4. Troubleshooting and conventions")
        col.label(text="   a. The mesh vertex positions must be in micrometers.")
        col.label(text="      After creating the mesh (for example by a marching cubes),")
        col.label(text="      make sure to convert from pixels to microns!")
        col.label(text="   b. When importing a mesh into blender, choose Y=Forward and Z=Up.")
        col.label(text="      This ensures the mesh coordinates match the image axes!")
        col.label(text="   c. Check the Info Monitor (Scripting window) for messages and errors.")
        
class TissueCartographyPanel(Panel):
    """Class defining layout of user interface (buttons, inputs, etc.)"""
    bl_label = "Tissue Cartography"
    bl_idname = "SCENE_PT_tissue_cartography"
    bl_space_type = 'PROPERTIES'
    bl_region_type = 'WINDOW'
    bl_context = "scene"

    def draw(self, context):
        layout = self.layout
        scene = context.scene

        layout.prop(scene, "tissue_cartography_file")
        layout.prop(scene, "tissue_cartography_resolution")
        layout.operator("scene.load_tiff", text="Load")
        layout.prop(scene, "tissue_cartography_offsets")
        layout.prop(scene, "projection_resolution")
        layout.operator("scene.create_projection", text="Create Projection")
        layout.operator("scene.help_popup", text="Help", icon='HELP')
        
def register():
    """Add the add-on to the blender user interface"""
    bpy.utils.register_class(LoadTIFFOperator)
    bpy.utils.register_class(CreateProjectionOperator)
    bpy.utils.register_class(TissueCartographyPanel)
    bpy.utils.register_class(HelpPopupOperator)
    
    bpy.types.Scene.tissue_cartography_file = StringProperty(
        name="File Path",
        description="Path to the TIFF file",
        subtype='FILE_PATH',
    )
    bpy.types.Scene.tissue_cartography_resolution = FloatVectorProperty(
        name="x/y/z Resolution (µm)",
        description="Resolution in microns along x, y, z axes",
        size=3,
        default=(1.0, 1.0, 1.0),
    )
    bpy.types.Scene.tissue_cartography_offsets = StringProperty(
        name="Normal Offsets (µm)",
        description="Comma-separated list of floats for multilayer projection offsets",
        default="0",
    )
    bpy.types.Scene.projection_resolution = IntProperty(
        name="Projection Resolution",
        description="Resolution for the projection (e.g., 1024 for 1024x1024)",
        default=1024,
        min=1,
    )

def unregister():
    bpy.utils.unregister_class(LoadTIFFOperator)
    bpy.utils.unregister_class(CreateProjectionOperator)
    bpy.utils.unregister_class(TissueCartographyPanel)
    bpy.utils.unregister_class(HelpPopupOperator)
    bpy.utils.unregister_class(OpenHelpWindowOperator)
    del bpy.types.Scene.tissue_cartography_file
    del bpy.types.Scene.tissue_cartography_resolution_array
    del bpy.types.Scene.tissue_cartography_data
    del bpy.types.Scene.tissue_cartography_offsets_array
    del bpy.types.Scene.tissue_cartography_baked_normals
    del bpy.types.Scene.tissue_cartography_baked_world_positions
    del bpy.types.Scene.tissue_cartography_baked_data

if __name__ == "__main__":
    register()

### baking normal and world position

Texture baking the surface normals and vertex positions tunrs out to be more involved than I would like.

In particular, it is difficult to guarantee that the coordinates are not distorted or rescaled in a way that I don't understand. Also, it can be slow. Also difficult to get to work as a script, unfortunately. Often I just get black images for reasons I do not understand.

I will therefore resort to using (manual) interpolation. Luckily, it appears that `scipy` is available in blender python. We need add one extra step (UV layout mask) though,

### Baking using custom interpolation

In [ ]:
import bpy
import numpy as np
import tifffile
from scipy import interpolate
from pathlib import Path

def load_png(image_path):
    """Load .png into numpy array."""
    image = bpy.data.images.load(image_path)
    width, height = image.size
    pixels = np.array(image.pixels[:], dtype=np.float32)
    return pixels.reshape((height, width, -1))

def get_uv_layout(uv_layout_path, image_resolution):
    """Get UV layout mask for currently active object and save UV layout to disk"""
    bpy.ops.object.mode_set(mode='EDIT')
    bpy.ops.uv.export_layout(filepath=uv_layout_path, size=(image_resolution, image_resolution), opacity=1)
    bpy.ops.object.mode_set(mode='OBJECT')
    UV_layout = load_png(uv_layout_path)
    return (UV_layout.sum(axis=-1) > 0)[::-1]

def get_uv_normal_world_per_loop(mesh_obj, filter_unique=False):
    """
    Get UV, normals, and world and normal for each loop (half-edge) as np.array.
    
    If filter_unique, remove "duplicate" loops (for which UV, normals and position
    are identical).
    """
    if not mesh_obj:
        raise TypeError("No object selected")
    if mesh_obj.type != 'MESH':
        raise TypeError("Selected object is not a mesh")
    world_matrix = mesh_obj.matrix_world
    uv_layer = mesh_obj.data.uv_layers.active
    if not uv_layer:
        raise RuntimeError("Mesh does not have an active UV map")
    loop_uvs = np.zeros((len(mesh_obj.data.loops), 2), dtype=np.float32)
    loop_normals = np.zeros((len(mesh_obj.data.loops), 3), dtype=np.float32)
    loop_world_positions = np.zeros((len(mesh_obj.data.loops), 3), dtype=np.float32)
    for loop in mesh_obj.data.loops:
        loop_uvs[loop.index] = uv_layer.data[loop.index].uv
        loop_normals[loop.index] = mesh_obj.data.vertices[loop.vertex_index].normal
        loop_world_positions[loop.index] = world_matrix @ mesh_obj.data.vertices[loop.vertex_index].co
    #loop_normals = np.round((loop_normals.T/np.linalg.norm(loop_normals, axis=1)).T, decimals=4)
    if filter_unique:
        unqiue_loops = np.unique(np.hstack([loop_uvs, loop_normals, loop_world_positions]), axis=0)
        loop_uvs, loop_normals, loop_world_positions = (unqiue_loops[:,:2], unqiue_loops[:,2:5], unqiue_loops[:,5:])
    return loop_uvs, loop_normals, loop_world_positions

def bake_per_loop_values_to_UV(loop_uvs, loop_values, image_resolution):
    """
    Bake (interpolate) values (normals or world position) defined per loop into the UV square.
    
    UV coordinates outside [0,1] are ignored.
    
    Parameters
    ----------
    loop_uvs : np.array of shape (n_loops, 2)
        UV coordinates of loop.
    loop_values : np.array of shape (n_loops, ...)
        Input field. Can be an array with any number of axes (e.g. scalar or vector field).
    image_resolution : int, default 256
        Size of UV grid. Determines resolution of result.

    Returns
    -------
    
    interpolated : np.array of shape (uv_grid_steps, uv_grid_steps, ...)
        Field across [0,1]**2 UV grid, with a uniform step size. UV positions that don't
        correspond to any value are set to np.nan.
            
    """
    U, V = np.meshgrid(*(2*(np.linspace(0,1, image_resolution),)))
    interpolated = interpolate.griddata(loop_uvs, loop_values, (U, V), method='linear')[::-1]
    return interpolated
    
image_resolution = 256
loop_uvs, loop_normals, loop_world_positions = get_uv_normal_world_per_loop(bpy.context.object, filter_unique=True)
baked_normals = bake_per_loop_values_to_UV(loop_uvs, loop_normals, image_resolution=image_resolution)
baked_normals = (baked_normals.T/np.linalg.norm(baked_normals.T, axis=0)).T
baked_world_positions = bake_per_loop_values_to_UV(loop_uvs, loop_world_positions, image_resolution=image_resolution)

# obtain UV layout and use it to get a mask
uv_layout_path = str(Path(bpy.path.abspath("//")).joinpath('UV_layout.png'))
mask = get_uv_layout(uv_layout_path, image_resolution)
baked_normals[~mask] = np.nan
baked_world_positions[~mask] = np.nan

# set as global variables and save to disk
bpy.types.Scene.tissue_cartography_baked_normals = baked_normals
bpy.types.Scene.tissue_cartography_baked_world_positions = baked_world_positions 
tifffile.imwrite(Path(bpy.path.abspath("//")).joinpath('NormalMapBarycentric.tif'), baked_normals)       
tifffile.imwrite(Path(bpy.path.abspath("//")).joinpath('PositionMapBarycentric.tif'), baked_world_positions)      

### Baking using render

In [ ]:
## Texture baking - here for the normals

import bpy
import numpy as np
import tifffile
from pathlib import Path

obj = bpy.context.object

# Check if the object is a mesh
if obj.type != 'MESH':
    raise TypeError("The active object must be a mesh.")
    
# Step 1: Create a new, empty material for the currently selected object

if "NormalMap" in bpy.data.materials:
    bpy.data.materials.remove(bpy.data.materials["NormalMap"])
material = bpy.data.materials.new(name="NormalMap")
material.use_nodes = True  # Enable nodes for the material
for node in material.node_tree.nodes:
    material.node_tree.nodes.remove(node)
# Assign the material to the object
obj.data.materials.append(material)
obj.active_material = material

# Step 2: Create and select an image texture with 512x512 pixels in the new material

if "TextureImage" in bpy.data.images:
    bpy.data.images.remove(bpy.data.images["TextureImage"])
image = bpy.data.images.new(name="TextureImage", width=512, height=512)
# Create the Image Texture node
image_texture_node = material.node_tree.nodes.new(type="ShaderNodeTexImage")
image_texture_node.image = image

# Step 3: Set the render engine to Cycles and set the bake settings

bpy.context.scene.render.engine = 'CYCLES'
bpy.context.scene.cycles.bake_type = 'NORMAL'
bpy.context.scene.render.bake.normal_space = 'OBJECT'
bpy.context.scene.render.bake.normal_r = 'POS_X'
bpy.context.scene.render.bake.normal_g = 'POS_Y'
bpy.context.scene.render.bake.normal_b = 'POS_Z'
bpy.context.scene.render.bake.margin = 1
bpy.context.scene.render.bake.use_clear = True

# Step 4: Bake the normal map (trigger the bake)
    
bpy.ops.object.bake()

# Step 5: assign pixel values to a numpy array (global variable) and save as .tif file

baked_image = bpy.data.images["TextureImage"]  # Access the baked image by name
pixels = np.array(baked_image.pixels)  # Get the pixel data as a 1D array
# Reshape the 1D array into a 2D array (image width x height x 4 channels)
pixels = pixels.reshape((baked_image.size[1], baked_image.size[0], 4))[:,:,:-1]
#pixels = (2*pixels - 1) # recale to [-1,1] and normalize 
#pixels = (pixels.T / np.linalg.norm(pixels.T, axis=0)).T

bpy.types.Scene.tissue_cartography_normal_map = pixels
tifffile.imwrite(Path(bpy.path.abspath("//")).joinpath('NormalMap.tif'), pixels)

In [ ]:
### Here for the position

import bpy
import numpy as np
import tifffile
from pathlib import Path



# Step 0: Check if the selected object is a mesh

obj = bpy.context.object
if obj.type != 'MESH':
    raise TypeError("The active object must be a mesh.")
    
# Step 1: Create a new, empty material for the currently selected object

if "PositionMap" in bpy.data.materials:
    bpy.data.materials.remove(bpy.data.materials["PositionMap"])
material = bpy.data.materials.new(name="PositionMap")
# Set up nodes
material.use_nodes = True 
for node in material.node_tree.nodes:
    material.node_tree.nodes.remove(node)
# Create nodes
geometry_node = material.node_tree.nodes.new(type="ShaderNodeNewGeometry")
vector_math_node = material.node_tree.nodes.new(type="ShaderNodeVectorMath")
vector_math_node.operation = 'MULTIPLY_ADD'  # Scale and offset
emission_node = material.node_tree.nodes.new(type="ShaderNodeEmission")
output_node = material.node_tree.nodes.new(type="ShaderNodeOutputMaterial")
# Position the nodes
geometry_node.location = (-300, 0)
vector_math_node.location = (0, 0)
emission_node.location = (300, 0)
output_node.location = (600, 0)
# Connect nodes
material.node_tree.links.new(geometry_node.outputs['Position'], vector_math_node.inputs[0])
material.node_tree.links.new(vector_math_node.outputs['Vector'], emission_node.inputs['Color'])
material.node_tree.links.new(emission_node.outputs['Emission'], output_node.inputs['Surface'])
# Set scaling (adjust these values based on your scene's scale)
vector_math_node.inputs[1].default_value = (0.1, 0.1, 0.1)  # Scale
vector_math_node.inputs[2].default_value = (0.5, 0.5, 0.5)  # Offset to center in [0, 1]
# Assign the material to the object
obj.data.materials.append(material)
obj.active_material = material

# Step 2: Create and select an image texture with 512x512 pixels in the new material

if "TextureImagePositionMap" in bpy.data.images:
    bpy.data.images.remove(bpy.data.images["TextureImagePositionMap"])
image = bpy.data.images.new(name="TextureImagePositionMap", width=512, height=512)
# Create the Image Texture node
image_texture_node = material.node_tree.nodes.new(type="ShaderNodeTexImage")
image_texture_node.image = image
image_texture_node.location = (-600, -200)

# Step 3: Set the render engine to Cycles and set the bake settings

bpy.context.scene.render.engine = 'CYCLES'
bpy.context.scene.cycles.bake_type = 'EMIT'
bpy.context.scene.render.bake.margin = 1
bpy.context.scene.render.bake.use_clear = True


# Step 4: Bake the normal map (trigger the bake)

bpy.context.view_layer.objects.active = obj
bpy.ops.object.bake()

# Step 5: assign pixel values to a numpy array (global variable) and save as .tif file

# Save the image
image.filepath_raw = "//position_map.png"
image.file_format = 'PNG'
image.save()

baked_image = bpy.data.images["TextureImagePositionMap"]  # Access the baked image by name
pixels = np.array(baked_image.pixels)  # Get the pixel data as a 1D array
# Reshape the 1D array into a 2D array (image width x height x 4 channels)
pixels = pixels.reshape((baked_image.size[1], baked_image.size[0], 4))[:,:,:-1]
#pixels = (2*pixels - 1) # recale to [-1,1] and normalize 
#pixels = (pixels.T / np.linalg.norm(pixels.T, axis=0)).T

bpy.types.Scene.tissue_cartography_positon_map = pixels
tifffile.imwrite(Path(bpy.path.abspath("//")).joinpath('PositionMap.tif'), pixels)



In [ ]:
# creating ortho-slices.

import bpy
import numpy as np
from skimage import transform

def create_slice_plane(length, width, height, axis='z', position=0.0):
    """
    Creates a 2D plane as a slice of a rectangular box along a specified axis.
    The plane lies within the bounds of the box.

    Args:
        length (float): Length of the box along the X-axis.
        width (float): Width of the box along the Y-axis.
        height (float): Height of the box along the Z-axis.
        axis (str): Axis along which to slice ('x', 'y', or 'z').
        position (float): Position along the chosen axis for the slice plane.
                          Should be within the range of the box dimensions.
    """
    current_active = bpy.context.active_object
    # Validate axis and position
    if axis not in {'x', 'y', 'z'}:
        raise ValueError("Axis must be 'x', 'y', or 'z'.")
    
    axis_limits = {'x': length, 'y': width, 'z': height}
    if not (0.0 <= position <= axis_limits[axis]):
        raise ValueError(f"Position must be within [0, {axis_limits[axis]}] for axis {axis}.")

    # Create the plane's dimensions based on the slicing axis
    if axis == 'x':
        plane_size = (width, height)
        location = (position, width / 2, height / 2)
        rotation = (0, 1.5708, 0)  # Rotate to align with the YZ-plane
    elif axis == 'y':
        plane_size = (length, height)
        location = (length / 2, position, height / 2)
        rotation = (1.5708, 0, 0)  # Rotate to align with the XZ-plane
    else:  # 'z'
        plane_size = (length, width)
        location = (length / 2, width / 2, position)
        rotation = (0, 0, 0)  # No rotation needed for the XY-plane

    # Add a plane
    bpy.ops.mesh.primitive_plane_add(size=2, location=(0, 0, 0))
    plane = bpy.context.active_object
    plane.name = f"SlicePlane_{axis.upper()}_{position:.2f}"

    # Scale and position the plane
    plane.scale = (plane_size[0] / 2, plane_size[1] / 2, 1)
    plane.location = location
    plane.rotation_euler = rotation

    # Apply transformations (scale, location, rotation)
    bpy.ops.object.transform_apply(location=True, scale=True, rotation=True)

    # Restore the previously active object
    if current_active:
        bpy.context.view_layer.objects.active = current_active

    return plane


def create_material_from_array(slice_plane, array, material_name="SliceMaterial"):
    """
    Creates a material for a plane using a 2D numpy array as a texture.

    Args:
        slice_plane (bpy.types.Object): The plane object to which the material will be applied.
        array (numpy.ndarray): 2D array representing grayscale values (0-1), or 3D array representing RGBA values (0-1).
        material_name (str): Name of the new material.
    """
    # Validate input array
    if not len(array.shape) in [2,3]:
        raise ValueError("Input array must be 2D.")
    
    # Normalize array to range [0, 1] and convert to a flat list
    image_height, image_width = array.shape[:2]
    pixel_data = np.zeros((image_height, image_width, 4), dtype=np.float32)  # RGBA
    if len(array.shape) == 2:
        pixel_data[..., 0] =  pixel_data[..., 1] = pixel_data[..., 2] = array
        pixel_data[..., 3] = 1.0  # Alpha
    else:
        pixel_data[...] = array
    pixel_data = pixel_data.flatten()

    # Create a new image in Blender
    image = bpy.data.images.new(name="SliceTexture", width=image_width, height=image_height)
    image.pixels = pixel_data.tolist()

    # Create a new material
    material = bpy.data.materials.new(name=material_name)
    material.use_nodes = True
    nodes = material.node_tree.nodes
    links = material.node_tree.links

    # Clear default nodesx
    for node in nodes:
        nodes.remove(node)

    # Add required nodes
    texture_node = nodes.new(type="ShaderNodeTexImage")
    texture_node.image = image
    bsdf_node = nodes.new(type="ShaderNodeBsdfPrincipled")
    output_node = nodes.new(type="ShaderNodeOutputMaterial")

    # Arrange nodes
    texture_node.location = (-400, 0)
    bsdf_node.location = (0, 0)
    output_node.location = (400, 0)

    # Connect nodes
    links.new(texture_node.outputs["Color"], bsdf_node.inputs["Base Color"])
    links.new(bsdf_node.outputs["BSDF"], output_node.inputs["Surface"])

    # Assign the material to the plane
    slice_plane.data.materials.append(material)

# Example usage: Create a slice of a box with dimensions 4x5x6 along the Z-axis at position 3

length, width, height = (np.array(bpy.context.scene.tissue_cartography_data.shape[1:]) *
                         bpy.context.scene.tissue_cartography_resolution_array)

slice_plane = create_slice_plane(length, width, height, axis='z', position=50)



slice_img = bpy.context.scene.tissue_cartography_data[0,:,:,50] # need to 
slice_img = slice_img-np.quantile(slice_img, 0.05)
slice_img = slice_img/np.quantile(slice_img, 0.95)
slice_img = np.clip(slice_img, 0, 1)
slice_img = slice_img.T


create_material_from_array(slice_plane, slice_img)




#create_box_from_cube(190, 509, 188)
